In [12]:
import os
import re
import gensim
import networkx as nx
import logging
import pandas as pd
import urllib.request
from gensim.models import word2vec
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot') 
import sqlite3
import csv
from flask import Flask
from flask import url_for, render_template, request, redirect

In [13]:
def champion(area):
    record = 0
    for id, result in area.items():
        if result >= record:
            record = result
            champion = id

In [14]:
base = 'ruscorpora_mystem_cbow_300_2_2015.bin.gz'
if base not in os.listdir('.'):
    urllib.request.urlretrieve("http://rusvectores.org/static/models/rusvectores2/ruscorpora_mystem_cbow_300_2_2015.bin.gz", "ruscorpora_mystem_cbow_300_2_2015.bin.gz")

In [15]:
base = 'ruscorpora_mystem_cbow_300_2_2015.bin.gz'
model = gensim.models.KeyedVectors.load_word2vec_format(base, binary=True)

MemoryError: 

In [16]:
def semgraph(core):
    subcore = []
    outskirts = []
    G = nx.Graph()
    G.add_node(core)
    if core in model:
        for i in model.most_similar(positive=[core], topn=5):
            print(i[0], i[1])
            friend = i[0]
            if "_S" in friend and model.similarity(core, friend) >= 0.5:
                subcore.append(friend)
            print('\n')
    for element in subcore:
        print(element)
        G.add_node(element)
        G.add_edge(core, element, weight = model.similarity(core, element))
        for ii in model.most_similar(positive=[element], topn=5):
            print(ii[0], ii[1])
            acqui = ii[0]
            if "_S" in acqui and model.similarity(element, acqui) >= 0.5:
                outskirts.append(acqui)
            print('\n')
    for item in outskirts:
        G.add_node(item)
        if model.similarity(core, item) >= 0.5:
            G.add_edge(core, item, weight = model.similarity(core, item))
        for element in subcore:
            if model.similarity(element, item) >= 0.5:
                G.add_edge(element, item, weight = model.similarity(element, item))
    report = "The most degree-central is : " + champion(nx.degree_centrality(G)) + "/n"
    + "The most betweenness-central is: " + champion(nx.betweenness_centrality(G)) + "/n"
    + "The most closeness-central is: " + champion(nx.closeness_centrality(G)) + "/n"
    + "The most eigenvector-central is: " + champion(nx.eigenvector_centrality(G)) + "/n"
    + "Graph radius is: " + str(nx.radius(G)) + "/n"
    + "Graph diametre is : " + str(nx.diameter(G)) + "/n"
    + "Graph assortativity coefficient is: " + str(nx.degree_pearson_correlation_coefficient(G)) + "/n"
    + "Graph clusterization coefficient is:" + str(nx.average_clustering(G)) + "/n"
    + "Graph density is: " + str(nx.density(G))
    return report
print(semgraph("народ_S"))
    pos=nx.spring_layout(G)
    nx.draw_networkx_nodes(G, pos, node_color='red', node_size=5) 
    nx.draw_networkx_edges(G, pos, edge_color='yellow')
    nx.draw_networkx_labels(G, pos, font_size=20, font_family='Arial')
    plt.axis('off') 
    plt.show()

MemoryError: 

In [20]:
app = Flask(__name__)
@app.route('/')
def index():
    urls = {'главная (эта страница)': url_for('index'),
            'форма': url_for('form')}
    return render_template('index.html', urls=urls)
@app.route('/form')
def form():
    return render_template('form.html')
@app.route('/results')
def result():
    if request.args:
        query = request.args['query']
        report = semgraph(query)
        return render_template('results.html', query=query)
    return redirect(url_for('questionnaire'))
if __name__ == '__main__':
    import os
    app.debug = True
    port = int(os.environ.get("PORT", 5000))
    app.run(host='0.0.0.0', port=port)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with stat


SystemExit: 1

C:\Users\User\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3275: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
